#### Este Projeto tem como objetivo obter o conteudo de artigos da internet, armazenar seu conteudo em um banco de dados, fazer tratativas, criar nuvens de palavras, mostrar verbos e nomes relevantes , em seguida fazendo trinamentos para identificar a natureza das frases do texto, como positivas, negativas ou neutras

#### This project has the purpose of geting the content of a few internet articles, store said contents in a database, then treat it, create a wordcloud, show verbs and relevant names, then train it to identify the nature of the text's sentences, as positive, negative or neutr

In [72]:
## importando bibliotecas
## importing Libraries

from urllib.request import urlopen
from bs4 import BeautifulSoup
import bs4
import urllib.request as urllib_request
import pandas as pd
import pymongo
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud
from os import path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS



In [2]:
## função responsavel por remover residuos do codigo html e partes indesejadas de texto dos resultados do scraping
## function responsible for removing html residue and unwanted bits of text from the scrapping results

def tratativa_um (paragrafos):
  paragrafos = paragrafos.replace('\n', '').replace('\r', '').replace('\t', '').replace('<p>','').replace('</p>','').replace('      Primeira Página da Folha de', '').replace('             - Reprodução', '')
  paragrafos = paragrafos.lower()
  return paragrafos

In [29]:
## RODAR APENAS UMA VEZ!!# baixando arquivo spacy para npl
## RUN ONLY ONCE!! downloading spacy file for npl
#!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 59.3MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186282 sha256=8be0c9286ef9ce681c49b9daec3c7f1160edac1c4c783c93ed9c1140dee96198
  Stored in directory: /tmp/pip-ephem-wheel-cache-xns_2t4l/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
## criando o banco de dados
## creating Database

# cliente = pymongo.MongoClient("mongodb://localhost:27017/")
# artigos_db = cliente["Artigos_Database"]
# artigos_col = artigos_db["Artigos"]

In [80]:
def tratativa_dois(texto):
    nlp = spacy.load("pt_core_news_sm")
    texto = nlp(texto)
    
    tokens_validos = []
    for token in texto:
      nonstop = not token.is_stop and token.is_alpha
      if nonstop:
             tokens_validos.append(token.text)      

    return " ".join(tokens_validos)


In [70]:
def word_cloud(text):
    stopwords = set(STOPWORDS)
    wc = WordCloud(width=800, height=500, background_color="black", max_words=2000,stopwords=stopwords, contour_width=3, contour_color='green').generate(text)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
# def trata_textos(doc):
#     tokens_validos = []
#     for token in doc:
#         e_valido = not token.is_stop and token.is_alpha
#         if e_valido:
#             tokens_validos.append(token.text)

#     if len(tokens_validos) > 2:
#         return " ".join(tokens_validos)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Sprint 3/Tweets_Mg.csv", encoding='utf-8')
df_treino = df[["Text", "Classificacao"]]
# df_treino.head()

nlp = spacy.load("pt_core_news_sm")
doc = nlp(str(df_treino[:5500]))

textos_para_tratamento = (titulos.lower() for titulos in df_treino["Text"])
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000, n_process = -1)]


In [ ]:
df_treino.insert(1, "tratado", textos_tratados)

df_treino = df_treino.dropna()
df_treino = df_treino.drop_duplicates(subset='tratado', keep='first')


In [ ]:
df_treino

,Text,tratado,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,catedral santo antônio governador valadares mg,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",governador valadares minas gerais,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,psol questionar aumento vereadores e prefeito ...,Negativo
5,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",bandido morto deputado cabo júlio condenado e ...,Neutro
6,"""..E 25% dos mineiros dizem não torcer para ti...",e mineiros torcer time nenhum e cruzeiro pq,Neutro
...,...,...,...
8173,"Trio é preso suspeito de roubo, tráfico e abus...",trio preso suspeito roubo tráfico e abuso sexu...,Positivo
8186,"Trio é preso suspeito de roubo, tráfico e abus...",trio preso suspeito roubo tráfico e abuso sexu...,Positivo
8191,"Trio é preso suspeito de roubo, tráfico e abus...",trio preso suspeito roubo tráfico e abuso sexu...,Positivo
8197,"Trio é preso suspeito de roubo, tráfico e abus...",trio preso suspeito roubo tráfico e abuso sexu...,Positivo


In [ ]:
# train_x = pd.DataFrame(df_treino['tratado'])
# train_y = pd.DataFrame(df_treino['Classificacao'])

# model = LogisticRegression()
# model.fit(train_x, train_y)


ValueError: ignored

In [95]:
## Gerando uma lista contendo o conteudo dos artigos, pegando os textos a partir da url

url = [
       'https://www1.folha.uol.com.br/banco-de-dados/2021/02/1971-astronautas-recebem-autorizacao-da-nasa-para-tentar-o-pouso-na-lua.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2021/01/1971-governo-afirma-que-troca-com-sequestradores-nao-vai-virar-rotina.shtml',
       ]

lista = []

for noticia in url:
    response = urlopen(noticia)
    html = response.read()

    soup = BeautifulSoup(html, 'html.parser')
    paragrafos = soup.find('div', class_='c-news__body').getText()
    titulo = soup.title.string
    p_um = tratativa_um(paragrafos)
    artigo = {'titulo':titulo, 'paragrafos':p_um}
    # artigos_col.insert_one(artigo)
    p_dois = tratativa_dois(p_um)
    print(artigo)
    print(p_dois)
    #word_cloud(str(p_dois))
    lista.append([p_dois])
    

print(lista)


{'titulo': '1971: Astronautas recebem autorização da Nasa para tentar o pouso na Lua - 01/02/2021 - Banco de Dados - Folha', 'paragrafos': 'os astronautas da apollo 14 foram formalmente autorizados pela nasa a pousar na lua na sexta-feira (5).o aval foi dado apesar de o sistema de junção entre os módulos de comando e o lunar ter apresentado dificuldades para fazer a união. várias tentativas foram feitas para conseguir a conexão.com as naves já firmemente ligadas, os astronautas desmontaram o mecanismo de acoplamento e o submeteram a vários testes. nenhum defeito foi encontrado. "não entendo absolutamente. tudo funciona perfeitamente", declarou o comandante da missão da apollo 14, alan shepard jr. 2 de fevereiro de 1971'}
astronautas apollo formalmente autorizados nasa a pousar lua aval dado apesar o junção módulos comando e o lunar apresentado dificuldades a união várias tentativas feitas conseguir a naves firmemente ligadas astronautas desmontaram o mecanismo acoplamento e o submetera

In [103]:
nlp = spacy.load("pt_core_news_sm")
textos = nlp(str(lista))

verbos = []
for token in textos:
  if token.pos_ == 'VERB' and token.is_alpha == True and token.orth_ not in verbos:
    verbos.append(token.orth_)

verbos 

['apollo',
 'autorizados',
 'pousar',
 'dado',
 'apresentado',
 'feitas',
 'conseguir',
 'ligadas',
 'desmontaram',
 'submeteram',
 'encontrado',
 'entendo',
 'funciona',
 'declarou',
 'divulgou',
 'dizendo',
 'adotado',
 'aceitar',
 'soltar',
 'estavam',
 'viajaram',
 'chamados',
 'significa',
 'tornar',
 'sequestrado',
 'informou']

In [104]:
entidades = []
for token in texto.ents:
  if token.orth_ not in entidades:
    entidades.append(token.orth_)
entidades

['astronautas apollo',
 'nasa',
 'lua',
 'união',
 'apollo alan shepard',
 'giovanni enrico bucher',
 'chile']

In [94]:

df = pd.DataFrame(lista)
df = df.rename(columns={0:'Paragrafo'})
df


AttributeError: ignored

In [ ]:
# dados = df.to_dict('list')
# valor = ((dados))
# artigos_col.insert_one(valor)


In [ ]:
# for x in artigos_col.find():
#     print(x)

In [ ]:
# for x in artigos_col.find({},{"_id":0,"Paragrafo":1}):
#     retorno = x

# df = pd.DataFrame(retorno)

# df